In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [2]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [7]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = ""
    consumer_secret = ""
    access_token = ""
    access_token_secret = ""

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-11.908419, 36.311530, 41.544370, 71.548697]  # Europe


    stream_listener = StreamListener(time_limit=300, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, track=['fake news'], languages=['en'], encoding="utf-8")

{'id': 1516250739894632458, 'username': 'TorreRellotge', 'created_at': 'Tue Apr 19 03:02:01 +0000 2022', 'lng': 1.893817, 'lat': 41.545639, 'text': 'DONG DONG DONG DONG DONG!!!!!'}
{'id': 1516250750275534859, 'username': 'ShortySan1986', 'created_at': 'Tue Apr 19 03:02:04 +0000 2022', 'lng': -2.772015, 'lat': 55.428405, 'text': "Looks like it's happening!!! https://t.co/N4AxLtIRZv"}
{'id': 1516250759620354051, 'username': 'kuzukito', 'created_at': 'Tue Apr 19 03:02:06 +0000 2022', 'lng': -9.0333065, 'lat': 51.903863, 'text': 'Do you remember when you joined Twitter? I do! #MyTwitterAnniversary https://t.co/A5GHCxsuGD'}
{'id': 1516250814049882114, 'username': 'KrisMonkeyDay', 'created_at': 'Tue Apr 19 03:02:19 +0000 2022', 'lng': -1.423629, 'lat': 54.881608, 'text': '😂😂 ha ha. Creased at this like.'}
{'id': 1516250860266864648, 'username': 'bishopstonwthr', 'created_at': 'Tue Apr 19 03:02:30 +0000 2022', 'lng': -2.596979, 'lat': 51.478303, 'text': '03:56 BST: Temperature: 6.3°C, Wind: S

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
